Peak Extraction

In [1]:
import physics
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter


df = pd.read_excel('../Standard calibration in culture media_extended.xlsx', sheet_name='Raw data', )
df.head()

# extracting potential and current values
concentrations = [0, 100, 100, 100, 50, 50, 50, 25, 25, 25, 15, 15, 15, 10, 10, 10, 7.5, 7.5, 7.5, 5, 5, 5, 2.5, 2.5, 1, 1, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25, 0.25, 0.25, 0.1, 0.1, 0.1, 0.1, 0.1]

# the same potential was applied to each experiment
potential: np.ndarray = df.iloc[1:, 0].values.astype(float)

# the next columns represent the currents for each experiment
# they will be stored inside a list
currents: list[np.ndarray] = []
for col in df.columns[1:]:
    currents.append(df[col].values[1:].astype(float))


Normalizing signal with the base signal

In [2]:
base_signal = currents[0]
normalized_currents: list[np.ndarray] = []
window_length = 5
polyorder = 3

for current in currents:
    normalized_current = current - base_signal
    smoothed_current = savgol_filter(normalized_current, window_length=window_length, polyorder=polyorder)
    normalized_currents.append(smoothed_current)

In [3]:
current1 = normalized_currents[5]
current2 = normalized_currents[10]
current3 = normalized_currents[15]
current4 = normalized_currents[32]
current5 = normalized_currents[39]
current6 = normalized_currents[40]
current7 = normalized_currents[38]


In [26]:
np.where((potential >= -0.45) & (potential <= -0.25))

(array([32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
        49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
        66, 67, 68, 69, 70, 71, 72], dtype=int64),)

In [20]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=potential[:140], y=current7[:140], mode="lines", name="signal"))
fig.add_trace(go.Scatter(x=potential[41:64], y=current7[41:64], mode="lines", name="signal", fill="tozeroy"))
fig.add_trace(go.Scatter(x=[potential[32]], y=[current7[32]], mode="lines+markers", name="points"))
fig.add_trace(go.Scatter(x=[potential[72]], y=[current7[72]], mode="lines+markers", name="points"))
fig.add_trace(go.Scatter(x=[potential[41]], y=[current7[41]], mode="lines+markers", name="left min"))
fig.add_trace(go.Scatter(x=[potential[63]], y=[current7[63]], mode="lines+markers", name="right min"))
fig.update_layout(
    # width=750,
    height=750,
    template="plotly_white"
)
fig.update_xaxes(
    showgrid=True,
    minor=dict(showgrid=True)
)
fig.update_yaxes(
    showgrid=True,
    minor=dict(showgrid=True)
)
fig.show()

In [16]:
def get_peak(E: np.ndarray, I: np.ndarray, win: tuple=(-4.5, -2.5)) -> float:
    E_start, E_end = win
    peak = np.where((potential >= -0.45) & (potential <= -0.25))
    i_start = peak[0][0]
    i_end = peak[0][-1]
    print(i_start)
    print(i_end)
    peak_i = np.argmax(I[i_start:i_end]) + i_start
    print(peak_i)
    print(I[peak_i])
    i_left = peak_i
    min_left = I[peak_i]
    i = peak_i
    while i_left > i_start and min_left > 0:
        if I[i] < min_left:
            min_left = I[i]
            i_left = i
        i -= 1
    print(i_left)
    print(min_left)
    print(I[i_left])
    
    i_right = peak_i
    min_right = I[peak_i]
    i = peak_i
    while i_right < i_end and min_right > 0:
        if I[i] < min_right:
            min_right = I[i]
            i_right = i
        i += 1
    print(i_right)
    print(min_right)
    print(I[i_right])
    
get_peak(potential, current7)

32
72
52
0.01859353881416744
41
-0.00018529367729370886
-0.00018529367729370886
63
-0.0001578967349837421
-0.0001578967349837421
